If you learned another object-oriented language before Python, you may have found it strange to use __len(collection)__ instead of __collection.len()__. 

The Python interpreter invokes special methods to perform basic object operations, often trigger by special syntax. The special method names are always writtem with leading and trailing double underscores (i.e., \____getitem__\__). For example, the syntax obj[key] is supported by the \____getitem__\__ special method. In order to evaluate my_collection[key], the interpreter calls my_collection.\____getitem__\__(key)

# A Pythonic Card Deck

The following is a very simple example, but it demonstrates the power of implementing just two special methods, \____getitem__\__ and \____len__\__

In [ ]:
import collections

# namedtuple can be used to build classes of just attributes
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [ ]:
beer_card = Card('7', 'diamonds')
beer_card

Although FrenchDeck is short, it packs a punch. A deck responds to the len() function by returing the number cards in it:

In [ ]:
deck = FrenchDeck()
len(deck)

Reading specific cards from the decks - say, the first or the last - should be easy as deck[0] or deck[-1], and this is what the \____getitem__\__ method provides:

In [ ]:
deck[0]

In [ ]:
deck[-1]

Should we create a method to pick a random card? No need. Python already has a function to get a random item from a sequence: random.choice. We can just use it on a deck instance:

In [ ]:
from random import choice
choice(deck)

In [ ]:
choice(deck)

In [ ]:
choice(deck)

We see two advantages of using special methods to leverage Python data model:
* The users of you classes don't have to memorize arbitrary method names for standard operations ("How to get the number of items? Is it .size(), .length(), or what?")
* It's easier to benefit from the rich Python standard library and void reinventing the wheel, like the random.choice function.

But it gets better. Because our \____getitem__\__ delegates to the [] operator of *self._cards*, our deck automatically supports slicing. Here is how we look at the top three cards from a brand new deck, and the pick just the aces by starting on index 12 and skipping 13 cards at a time:

In [ ]:
deck[:3]

In [ ]:
deck[12::13]

Just by implementing the __getitem__ special method, our deck is also iterable:

In [ ]:
for card in deck:
    print(card)

The deck can also be iterated in reverse:

In [ ]:
for card in reversed(deck):
    print(card)

Iteration is often implicit. If a collection has no \____contains__\__ method, the in operator does a sequential scan. Case in point: in words in our FrenchDeck class because it is iterable. Check it out:

In [ ]:
Card('Q', 'hearts') in deck

In [ ]:
Card('7', 'beasts') in deck

How about sorting? A common system of ranking cards is by rank (with aces being highest), then by suit in the order of spades (highest), then hearts, diamonds, and clubs (lowest). Here is a function that ranks cards by that rule, return 0 for 2 of clubs and 51 for aces of spades:

In [ ]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

Given spades_high, we can now list our deck in order of increasing rank:

In [ ]:
for card in sorted(deck, key=spades_high):
    print(card)

Although FrenchDeck implicitly inherits from *__object__*, its functionality is not inherited, but comes from leveraging the data model and composition. By implementing the special methods \____len__\__ and \____getitem__\__, our FrenchDecj behaves like a standard Python sequence, allowing it to benefit from core language features (e.g., iteration and slicing) and from the standard library, as shown by the examples using __random.choice__, __reverse__, and __sorted__. Thanks to composition, the \____len__\__ and \____getitem__\__ implementations can hand off all the work to a *list object*, *self._cards* 

# How Special Methods Are Used

The first thing to know about special methods is that they care meant to be called by the Python interpreter, and not by you. You don't write *my_object.\__len\__()*. You write *len(my_object)* and, if *my_object* is an instance of a user-defined class, then Python calls the *\__len\__* instance method you implemented. 

But for built-in types like *list*, *str*, *bytearray*, and so on, the interpreter takes a shortcut: the CPtyhon implementation of *len()* actually returns the value of the *ob_size* field in the *PyVarObject C* struct represents any variable-sized built-in object in memory. This is much faster than calling a method
More often than not, the special method call is implicit. For example, the statement for i in x: actually causes the invocation of *iter(x)*, which in turn may call *x.\__iter\__()* if that is available.

Normally, your code should not have many direct calls to special methods. Unless you are doing a lot of metaprogramming, you should be implementing special methods more often than invoking them explicitly. The only special method that is frequently called by user code directly is *\__init\__*, to invoke the initializer of the superclass in your own *\__init\__* implementation.

If you need to invoke a special method, it is usually better to call the related built-in function (e.g., len, iter, str, etc). These built-ins call the corresponding special method, but often provide other services and - for bultt-in types - are faster than method calls.

# Emulating Numeric Type

We will implement a class to represent two dimensional vectors - that is Euclidean vectors like those used in math and physics.

In [ ]:
from math import hypot

class Vector:
    
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        # %r means standard representation of the attribute
        return 'Vector(%r, %r)' % (self.x, self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [ ]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v1 + v2

In [ ]:
v = Vector(3, 4)
abs(v)

In [ ]:
v * 3

In [ ]:
abs(v * 3)

# Chapter Summary

By implementing special methods, your objects can behave like the built-in types, enabling the expressive coding style the community considers Pythonic.”

A basic requirement for a Python object is to provide usable string representations of itself, one used for debugging and logging, another for presentation to end users. That is why the special methods \____repr__\__ and \____str__\__ exist in the data model. 

Emulating sequences, as shown with the FrenchDeck example, is one of the most widely used applications of the special methods. Making the most of sequence types is the subject of Chapter 2, and implementing your own sequence will be covered in Chapter 10 when we create a multidimensional extension of the Vector class.

Thanks to operator overloading, Python offers a rich selection of numeric types, from the built-ins to decimal.Decimal and fractions.Fraction, all supporting infix arithmetic operators. Implementing operators, including reversed operators and augmented assignment, will be shown in Chapter 13 via enhancements of the Vector example.

The use and implementation of the majority of the remaining special methods of the Python data model are covered throughout this book. 